In [ ]:
from discovery_utils.getters import gtr
from discovery_utils.getters import crunchbase
from discovery_utils.utils import search

from src import PROJECT_DIR
from src import VECTOR_DB_DIR

OUTPUT_DIR = PROJECT_DIR / 'data/2025_01_MS_ahl/'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
import pandas as pd

In [ ]:
from discovery_utils.utils.llm.batch_check import LLMProcessor, generate_system_message


In [ ]:
CB = crunchbase.CrunchbaseGetter(vector_db_path=VECTOR_DB_DIR)

In [ ]:
SearchCB = search.SearchDataset(CB, CB.organisations_enriched, "config_MS_glp1.yaml")

In [ ]:
search_cb_df = SearchCB.do_search()

In [ ]:
search_cb_df[['id', 'name', 'short_description', 'homepage_url', '_score_keywords', '_score_vectors', '_score_avg']]


In [ ]:
relevant_df = search_cb_df.query("_score_avg > 0.3")

In [ ]:
system_message = generate_system_message("config_MS_glp1.yaml")
fields = [
    {"name": "is_relevant", "type": "str", "description": "A one-word answer: 'yes' or 'no'."},
]

check_data = dict(zip(relevant_df['id'], relevant_df['short_description']))

processor = LLMProcessor(
    output_path=str(OUTPUT_DIR / "output_MS_glp1.jsonl"),
    system_message=system_message,
    session_name="mission_studio",
    output_fields=fields,
)

processor.run(check_data, batch_size=15, sleep_time=0.5)

In [ ]:
relevant_check_df = pd.read_json(OUTPUT_DIR / "output_MS_glp1.jsonl", lines=True)

In [ ]:
relevant_checked_df = relevant_df.merge(relevant_check_df[['id', 'is_relevant']], left_on='id', right_on='id', how='left')
relevant_checked_df.query("is_relevant == 'yes'")[['id', 'name', 'short_description', 'homepage_url', '_score_avg']]